# Probabilities for musical generation in python
This script contains some tools for generating sequences using Markov models that can be used in music generation scenarios.

In [19]:
import random
import numpy as np

#UTIL FUNCTIONS:

#Event List to PDF:
def event_list_2_pdf(event_list):
  events, counts = np.unique(event_list, return_counts=True, axis=0)
  events=[tuple(x) for x in events]
  pdf=counts/sum(counts)
  return events, pdf

######################################################################
#Select Event from PDF:
#Now lets select a random element from a PDF:

def select_event_from_pdf (events, pdf):
  event_indexes=range(len(events))
  #print(event_indexes)
  u= np.random.choice(event_indexes, 1, p=pdf)[0]
  #print('u= ', u)
  return events[u]

new_event = select_event_from_pdf(events, pdf)
print ("new event", new_event)

######################################################################
#Train PDF:
def train(input_sequence):
  observed_transitions={}
  for i in range(len(input_sequence)-1):
    if input_sequence[i] in observed_transitions:
      observed_transitions[input_sequence[i]].append(input_sequence[i+1])
    else:
      observed_transitions[input_sequence[i]]=[input_sequence[i+1]]
  
  # lets print the list-based dictionary
  print ('Observed transition dict: \n ',observed_transitions)
  print("----------------------------------")
  
  # now lets make compact verisons of the values
  pdf_dict={}
  for key in observed_transitions:
    #print (key, observed_transitions[key])
    #print (event_list_2_pdf(observed_transitions[key]))
    pdf_dict[key]=[event_list_2_pdf(observed_transitions[key])]
  # print the compact version of the dictionary
  print(pdf_dict)

  return pdf_dict



new event (0, 1)


# Exercise 1
Now that we know how to generate sequences at this low cost form previous observations, we would want to steer the generaiton process. For example, lets say that our events encode two variables (MIDI pitch, dynamic) and that we want to select the lowest dynamic possible because we want ourgenerated  sequence to be very calm. At each selection step we would like to make some adjustments to the PDF to increase the probability of events whose second variable is smaller. For example:

If our next events are  [(0, 1), (0, 3), (2, 1), (2, 2)] and their pdf is  [0.25 0.25 0.25 0.25], we would like to increase the probability of events (0,1) and (2,1) because their second variable (dynamic) is the smallest. This means that we would be interfeering with what was observed (each future event has 25% probability) and change it so the resulting pdf is 

[0.5, 0, 0,5,0] 

This means that the selction of the next event would only be made between events (0,1) and (2,1), and that they would have 50% chance to be selected. Either of them fits us because we want elements that have small dynamic (both have 1).

How do we create a function that would be called in [1] that would: 
 

1.   check which possible events we have at [1]
2.   Analize input cue. For example, the generate_from_pdf funciton would need a "dynamic" input variable to inform what kind of dynamic we are favouring (1,2 or 3). Lets say we would want the softest, so we would call the function with a third variable with value 1.
3.   create a multuplying_PDF array that multiplies each element of the pdf by a specific value. In our example that array would be [0.5,0,0.5,0]. And then normalize the PDF

    original_pdf [0.25, 0.25, 0.25, 0.25]

    multiply_pf [1,   0,    1,   0] # it is not normalized!

    resultin_pdf [0.25, 0, 0.25, 0]

    normaliz_pdf [0.5, 0, 0.5, 0]

4.  use the resulting, normalized pdf in the select_event_from_pdf function.


Summarizing:
The idea is to analyze the list of events and, given what we find in it and the expressed desire when calling the function, we create a probability function (can be unnormalized) to ajust the probabilities. 

The idea is that if we consistently apply this transformation at every generative step we would be systematicall guiding the sequence to be of some specific kind that we requested.

## Ideas for exercise 1
The shapes of the probability functions that we generate can have very diverse shapes. For example, they can be very extreme as the one above (1 or 0) or they can consider intermediate values. Lets see: The possible events we had were [(0, 1), (0, 3), (2, 1), (2, 2)] and we wanted to favor the lowest dynamics therefore we used 1 as input variable in the function. So we could have used a funtion that would generate the inverse of the observed dynamic: observed dynamics = [1,3,1,2] -> output probabilities = [2,0,2,1]. The funciton was:

(input - observed_dynamic) + 2

Lets try to make our own functions to process the pdfs







In [20]:
######################################################################
#GENERATE FROM PDF:

def generate_from_pdf(trained_pdf, N, cond):
  output_sequence=[]
  now=random.choice(list(trained_pdf.keys()))
  print (f'Paso 0:{now}')
  output_sequence.append(now)
  for x, i in enumerate(range(N-1)):

    next_events, next_pdf=trained_pdf[now][0]

    list_tmp= next_events
    print('lista2 temp', list_tmp)
    pdf_tmp= next_pdf
    print('pdf temp', pdf_tmp)

    
    #[1]

    pdf_cond = []
    if (cond==1): #We want to favour velocity=1
      for tupla in list_tmp:
        x= (tupla[1]-3)*(-1)
        pdf_cond.append(x)

    elif(cond==3): #We want to favour velocity=3
      for tupla in list_tmp:
        x= (tupla[1])-1
        pdf_cond.append(x)

    elif(cond==2): #We want to favour velocity=2
      for tupla in list_tmp:
        if (tupla[1]==2): 
          x= 2
          pdf_cond.append(x)
        else:
          x=1
          pdf_cond.append(x)
       
    print('PDF_COND', pdf_cond)  
    #After conditions!    
    pdf_new=[] #PDF_New= pdf
    pdf_mult = [pdf_tmp[i] * pdf_cond[i] for i in range(len(pdf_tmp))] #multiply original pdf with pdf_cond
    if (sum(pdf_mult)==0):
      pdf_new= pdf_tmp
    else:
      pdf_new = [elem/sum(pdf_mult) for elem in pdf_mult]#Normalyzing new PDF
    next= select_event_from_pdf(next_events,pdf_new)
    #[2]

    now=next
    print(f'STEP {i+1}: Prox events: {next_events}.  NEW pdf: {pdf_new}. --> {now} \n *+++++++++++++++++++++++++++++++++++++++++')
    output_sequence.append(now)
  
  return output_sequence

In [21]:
#TEST CASE:
######################################################################

input_sequence=[(0,1),(0,1),(2,1),(2,1),(2,1), (0,1), (0,3),(0,1),(2,2),(2,2),(2,3), (0,1)]

trained_pdf = train(input_sequence)
print(trained_pdf[(0,1)][0][1])

sequence = generate_from_pdf(trained_pdf, 6, 2)
print(f"Generated Sequence:  {sequence}")


Observed transition dict: 
  {(0, 1): [(0, 1), (2, 1), (0, 3), (2, 2)], (2, 1): [(2, 1), (2, 1), (0, 1)], (0, 3): [(0, 1)], (2, 2): [(2, 2), (2, 3)], (2, 3): [(0, 1)]}
----------------------------------
{(0, 1): [([(0, 1), (0, 3), (2, 1), (2, 2)], array([0.25, 0.25, 0.25, 0.25]))], (2, 1): [([(0, 1), (2, 1)], array([0.33333333, 0.66666667]))], (0, 3): [([(0, 1)], array([1.]))], (2, 2): [([(2, 2), (2, 3)], array([0.5, 0.5]))], (2, 3): [([(0, 1)], array([1.]))]}
[0.25 0.25 0.25 0.25]
Paso 0:(0, 3)
lista2 temp [(0, 1)]
pdf temp [1.]
PDF_COND [1]
STEP 1: Prox events: [(0, 1)].  NEW pdf: [1.0]. --> (0, 1) 
 *+++++++++++++++++++++++++++++++++++++++++
lista2 temp [(0, 1), (0, 3), (2, 1), (2, 2)]
pdf temp [0.25 0.25 0.25 0.25]
PDF_COND [1, 1, 1, 2]
STEP 2: Prox events: [(0, 1), (0, 3), (2, 1), (2, 2)].  NEW pdf: [0.2, 0.2, 0.2, 0.4]. --> (2, 2) 
 *+++++++++++++++++++++++++++++++++++++++++
lista2 temp [(2, 2), (2, 3)]
pdf temp [0.5 0.5]
PDF_COND [2, 1]
STEP 3: Prox events: [(2, 2), (2, 3)].  NE